# 🧠 Jupyter Notebook – Model z konfigurace `config_20250701_190425.json`

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    BatchNormalization, Activation
)
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0


# 🔁 Přizpůsob vstupní tvar dat
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))



In [ ]:
# Konfigurace modelu
config = {
    "global_config": {
        "name": "uz",
        "optimizer": "adam",
        "loss_function": "categorical_crossentropy",
        "metrics": "accuracy",
        "epochs": 12,
        "batch_size": 32,
        "augmentation": "FALSE"
    },
    "selected_dataset": {
        "family": "MNIST",
        "name": "mnist"
    },
    "layers": [
        {
            "type": "Input",
            "data": {
                "input_shape": [
                    28,
                    28,
                    1
                ]
            }
        },
        {
            "type": "Conv2D",
            "data": {
                "filters": 128,
                "kernel_size": "3",
                "stride": "1",
                "padding": "same",
                "activation": "ReLU"
            }
        },
        {
            "type": "MaxPooling2D",
            "data": {
                "pool_size": "3",
                "stride": "2",
                "padding": "same"
            }
        },
        {
            "type": "Conv2D",
            "data": {
                "filters": 128,
                "kernel_size": "5",
                "stride": "1",
                "padding": "same",
                "activation": "ReLU"
            }
        },
        {
            "type": "Dropout",
            "data": {
                "rate": 0.3
            }
        },
        {
            "type": "Conv2D",
            "data": {
                "filters": 64,
                "kernel_size": "5",
                "stride": "1",
                "padding": "same",
                "activation": "ReLU"
            }
        },
        {
            "type": "Flatten",
            "data": {
                "dummy": "flatten"
            }
        },
        {
            "type": "Dense",
            "data": {
                "units": 10,
                "activation": "ReLU"
            }
        }
    ]
}


In [ ]:
LAYER_TYPE_MAP = {
    "Input": lambda params: InputLayer(shape=tuple(params["input_shape"])),
    "Conv2D": lambda params: Conv2D(
        filters=int(params["filters"]),
        kernel_size=int(params["kernel_size"]),
        activation=params.get("activation", None),
        padding=params.get("padding", "valid")
    ),
    "MaxPooling2D": lambda params: MaxPooling2D(pool_size=int(params["pool_size"])),
    "Flatten": lambda params: Flatten(),
    "Dense": lambda params: Dense(
        units=int(params["units"]),
        activation=params.get("activation", None)
    ),
    "Dropout": lambda params: Dropout(rate=float(params["rate"])),
    "BatchNormalization": lambda params: BatchNormalization(),
    "Activation": lambda params: Activation(params["activation"])
}


In [ ]:
model = Sequential()

for layer_cfg in config["layers"]:
    layer_type = layer_cfg["type"]
    layer_params = layer_cfg["data"]
    if "activation" in layer_params and isinstance(layer_params["activation"], str):
        layer_params["activation"] = layer_params["activation"].lower()
    layer_fn = LAYER_TYPE_MAP.get(layer_type)
    if not layer_fn:
        raise ValueError(f"Neznámý typ vrstvy: {layer_type}")
    model.add(layer_fn(layer_params))

model.summary()


In [ ]:
# 🏃‍♂️ Kompilace a trénink modelu
train_cfg = config.get("global_config", {})

optimizer = train_cfg.get("optimizer", "adam")
loss = train_cfg.get("loss_function", "sparse_categorical_crossentropy")
metrics = [train_cfg.get("metrics", "accuracy")]
epochs = int(train_cfg.get("epochs", 10))
batch_size = int(train_cfg.get("batch_size", 32))
validation_split = float(train_cfg.get("validation_split", 0.2)) if "validation_split" in train_cfg else 0.2

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(
x_train, y_train,
epochs=epochs,
batch_size=batch_size,
validation_split=validation_split
)


In [ ]:
# 🧪 Vyhodnocení modelu
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"📊 Test loss: {test_loss:.4f}")
print(f"✅ Test accuracy: {test_accuracy:.4f}")
